# Zadanie 1

In [23]:
import plotly.express as px
import pandas as pd

dane = {
    'x1':   {1: 1, 2: 0, 3: 1, 4: 2},
    'x2':   {1: 0, 2: 1, 3: 1, 4: 0},
    'y':    {1: 2, 2: 2, 3: 3, 4: 4},
}

df = pd.DataFrame(dane)

fig = px.scatter_3d(
    data_frame=df,
    x='x1',
    y='x2',
    z='y',
    color='y',
    color_continuous_scale='Viridis',
    title='3D data'
)

fig.update_layout(
    scene=dict(
        xaxis=dict(title='x₁', backgroundcolor='black', gridcolor='#222', zerolinecolor='#444', color='white'),
        yaxis=dict(title='x₂', backgroundcolor='black', gridcolor='#222', zerolinecolor='#444', color='white'),
        zaxis=dict(title='y',  backgroundcolor='black', gridcolor='#222', zerolinecolor='#444', color='white'),
        bgcolor='black'
    ),
    width=1300,
    height=725,
    paper_bgcolor='black',
    font_color='white',
    title_x=0.5
)

print(df.head())
fig.show()


   x1  x2  y
1   1   0  2
2   0   1  2
3   1   1  3
4   2   0  4


In [24]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Dane
X = np.array([
    [1, 0],
    [0, 1],
    [1, 1],
    [2, 0]
])

y = np.array([2, 2, 3, 4])

# Model regresji liniowej
model = LinearRegression(fit_intercept=True)
model.fit(X, y)

# Współczynniki
print("β0 (intercept):", model.intercept_)
print("β1, β2 (coefficients):", model.coef_)






β0 (intercept): 0.75
β1, β2 (coefficients): [1.5 1. ]


In [25]:
import numpy as np
from sklearn.linear_model import Ridge

# Dodaj kolumnę jedynek (dla β0)
X_aug = np.c_[np.ones(len(X)), X]

# Policzymy Ridge bez interceptu (czyli dokładnie wzór (X^T X + λI)^(-1) X^T y)
ridge = Ridge(alpha=1.0, fit_intercept=False)
ridge.fit(X_aug, y)

b0, b1, b2 = ridge.coef_

print(f"β0: {b0:.4f}, β1: {b1:.4f}, β2: {b2:.4f}")

β0: 1.0000, β1: 1.2000, β2: 0.6000


In [26]:
from sklearn.linear_model import Lasso
import numpy as np

X = np.array([
    [1, 0],
    [0, 1],
    [1, 1],
    [2, 0]
])
y = np.array([2, 2, 3, 4])

model = Lasso(alpha=0.1, fit_intercept=True)
model.fit(X, y)

print("β0:", model.intercept_)
print("β1, β2:", model.coef_)

β0: 1.95
β1, β2: [0.8 0. ]


<div align="center">

![rysunek nr 1](images/rysunek_1.png)

### Opisz, jak metody regularyzacji wpływają na parametry modelu. Wytłumacz to na przykładzie Rysunku 1.

</div>

---

### **OLS (Ordinary Least Squares)**

Regresja liniowa OLS (Ordinary Least Squares) dopasowuje model w taki sposób, aby **zminimalizować błąd średniokwadratowy (MSE)** pomiędzy wartościami rzeczywistymi a przewidywanymi.  

Model szuka takich współczynników β, które najlepiej odwzorowują dane, bez żadnych ograniczeń.  

Funkcja kosztu przyjmuje postać:

$$
    J_{OLS}(β) = \sum_{i=1}^n (y_i - X_iβ)^2
$$

Na rysunku OLS znajduje się **w punkcie przecięcia elipsy błędu z osią**, który minimalizuje odległość od obserwacji.  
Nie ma żadnej kary za wielkość współczynników – dlatego współczynniki mogą przyjmować duże wartości, zwłaszcza gdy zmienne są silnie skorelowane (kolinearność).

---

### **Ridge (Regresja grzbietowa)**

Metoda Ridge dodaje do funkcji kosztu **karę L2**, która penalizuje duże wartości współczynników.  
Funkcja kosztu przyjmuje postać:

$$
    J_{Ridge}(β) = \sum_{i=1}^n (y_i - X_iβ)^2 + λ\sum_{j=1}^p β_j^2
$$

Oznacza to, że model oprócz dopasowania do danych musi również utrzymać współczynniki możliwie małe.  

Na rysunku granica regularyzacji Ridge ma kształt **okręgu (lub elipsy)**.  
Punkt przecięcia elipsy błędu z tym ograniczeniem znajduje się **bliżej zera**, dlatego współczynniki są **ściągane** (shrinkage), ale **nie stają się równe zeru**.  

W efekcie Ridge:
- zapobiega zbyt dużym wartościom współczynników,
- zmniejsza wariancję modelu,
- nie wykonuje selekcji zmiennych (żaden współczynnik nie znika całkowicie).

---

### **LASSO (Least Absolute Shrinkage and Selection Operator)**

LASSO wykorzystuje **karę L1**, czyli sumę wartości bezwzględnych współczynników:


$$
    J_{LASSO}(β) = \sum_{i=1}^n (y_i - X_iβ)^2 + λ\sum_{j=1}^p |β_j|
$$


W przeciwieństwie do Ridge, kara L1 ma kształt **romb (diament)** na rysunku.  
Ponieważ romb ma ostre wierzchołki, rozwiązanie często trafia **dokładnie na oś**, co oznacza, że niektóre współczynniki β stają się **równe zeru**.  

W ten sposób LASSO nie tylko ogranicza wielkość współczynników, ale też **usuwa nieistotne zmienne z modelu**.  
To sprawia, że model jest prostszy i łatwiejszy do interpretacji.

---

### **Porównanie wpływu regularyzacji na współczynniki (na podstawie Rysunku 1)**

| Metoda | Typ kary | Kształt ograniczenia | Efekt na współczynniki | Selekcja zmiennych |
|---------|-----------|----------------------|------------------------|--------------------|
| **OLS** | brak | brak | współczynniki mogą być bardzo duże | ❌ nie |
| **Ridge** | L2  | okrąg | współczynniki ściągane do zera, ale nigdy niezerowe | ❌ nie |
| **LASSO** | L1  | romb | część współczynników dokładnie równa zero | ✅ tak |

---

### 

### **Która metoda może być postrzegana jako metoda wyboru zmiennych?**

Metoda **LASSO** może być traktowana jako metoda wyboru zmiennych, ponieważ dzięki zastosowaniu kary L1 potrafi **całkowicie wyzerować niektóre współczynniki**.  
W efekcie usuwa z modelu zmienne, które nie wnoszą istotnej informacji — działa więc jednocześnie jako **regresja i selekcja cech**.

---


# Zadanie 2

# Zadanie 3

$$
    t = \frac{Estymator - Wartość \space Hipoteczna}{Błąd \space Standardowy \space Estymatora}
$$

$$
    t = \frac{\hat{\beta}_j - 0}{SE(\hat{\beta}_j)} = \frac{\hat{\beta}_j}{SE(\hat{\beta}_j)}
$$

$$
    tr(\frac{\alpha}{2}) \approx 2.04841
$$

$$
    t = \frac{\hat{\beta}_1}{SE(\hat{\beta}_1)} = \frac{0.8}{0.3} \approx 2.66667
$$

# Zadanie 4

# Zadanie 4 — Metryki klasyfikacji

Firma medyczna opracowuje model diagnostyczny do wykrywania rzadkiej choroby **X**.  
Choroba X występuje w około **2% populacji**. W badaniu walidacyjnym wzięło udział **1000 pacjentów**, wśród których **20 faktycznie miało chorobę**, a **980 było zdrowych**.  
Model przewiduje obecność choroby u **50 pacjentów**, z czego **15 faktycznie chorowało**, a **35 było zdrowych**.

---

## Macierz pomyłek (Confusion Matrix)

|                 | Choroba = Tak | Choroba = Nie |
|-----------------|:-------------:|:-------------:|
| **Predykcja = Tak** | **TP = 15**    | **FP = 35**    |
| **Predykcja = Nie** | **FN = 5**     | **TN = 945**   |

n = 1000, częstość choroby = 20/1000 = **2%**

---

## Metryki z definicjami i wynikami

- **Accuracy (dokładność)**  
Wzór:  

    $ Accuracy = \frac{TP + TN}{TP + TN + FP + FN} $

    Obliczenia:  
    ((15 + 945)/1000 = 0.96\) → **96%** 

---

- **Precision (precyzja, PPV)**  
  Wzór:  

    $  Precision = \frac{TP}{TP + FP}$

    Obliczenia:  
    \(15 / 50 = 0.30\) → **30%**

---

- **Recall (czułość, sensitivity)**  
  Wzór:  

  $ Recall = \frac{TP}{TP + FN} $

  Obliczenia:  
  \(15 / 20 = 0.75\) → **75%**

---

- **Specificity (swoistość)**  
  Wzór:  
  \[
  Specificity = \frac{TN}{TN + FP}
  \]  
  Obliczenia:  
  \(945 / 980 = 0.9643\) → **96.43%**

---

- **F1-score**  
  Wzór:  
  \[
  F1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}
  \]  
  Obliczenia:  
  \(2 \cdot \frac{0.30 \cdot 0.75}{0.30 + 0.75} = 0.4286\) → **42.86%**

---

## Interpretacja wyników

- **Accuracy 96%** wygląda wysoko, ale przy klasie rzadkiej (2%) jest **mylące**.  
- **Precision 30%**: tylko 3 na 10 „pozytywnych” alarmów to faktycznie chorzy → dużo fałszywych alarmów (FP).  
- **Recall 75%**: wykrywamy 3/4 chorych, 5 przypadków umyka (FN).  
- **Specificity ~96%**: dobrze rozpoznajemy zdrowych — wysoka wartość naturalna przy przewadze zdrowych.  
- **F1-score ~43%**: pokazuje umiarkowaną skuteczność modelu w diagnozowaniu chorych.

---

## Które metryki są najbardziej informatywne?

Przy **niezbalansowanych klasach** (rzadkie choroby) ważniejsze są **Precision, Recall i F1**, niż Accuracy.

- Jeśli celem jest **nie przegapić chorych** → patrzymy na **Recall**.  
- Jeśli celem jest **nie generować zbyt wielu fałszywych alarmów** → patrzymy na **Precision**.  
- **F1-score** stanowi kompromis między nimi.

Podobne sytuacje: wykrywanie oszustw (fraud detection), spamu, nowotworów, anomalii w danych lub defektów produkcyjnych.

---

## Trywialny klasyfikator

Klasyfikator, który **zawsze przewiduje „Nie choroba”**, osiągnie:

- **Accuracy = 980/1000 = 98%**  
- ale **Recall = 0** (nie wykrywa żadnego chorego)

To pokazuje, że **dokładność sama w sobie nie wystarcza** w przypadku silnie niezbalansowanych danych.

---

# Zadanie 5

$$
    Funkcja - f(x) = x^2 + 2x
$$

$$
    x_0 = 2
$$

$$
    η = 0.3
$$